In [ ]:
# DO NOT MODIFY
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import os
# import pandas as pd

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
CLIENT_SECRETS_FILE = './credentials/credentials.json' #/content/credentials.json
TOKEN_FILE = './credentials/token.json'#/content/token.json

data = []

def get_credentials():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    creds = flow.run_console()
    creds = flow.run_local_server(port=0)
    with open(TOKEN_FILE, 'w') as token:
        token.write(creds.to_json())
    return creds

def refresh_token(creds):
    if creds and creds.expired:
        creds.refresh(Request())
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        refresh_token(creds)
else:
    creds = get_credentials()

def get_google_analytics_data(view_id, start_date, end_date, dimension_name, metric_name, org_id):
    if creds and creds.expired:
        refresh_token(creds)

    analytics = build('analyticsreporting', 'v4', credentials=creds)

    request_body = {
        'reportRequests': [
            {
                'viewId': view_id,
                'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
                'dimensions': [{'name': 'ga:' + dimension_name}],
                'metrics': [{'expression': 'ga:' + metric_name}],
                'dimensionFilterClauses': [
                    {
                        'filters': [
                            {
                                'dimensionName': 'ga:landingPagePath',
                                'operator': 'PARTIAL',
                                'expressions': [org_id]
                            }
                        ]
                    }
                ]
            }
        ]
    }

    response = analytics.reports().batchGet(body=request_body).execute()

    if 'reports' in response:
      try:
        return response['reports'][0]['data']['rows']
      except:
        return 0
    else:
        print(f"Error: No data returned")
        return None

# df = pd.read_excel(excel_path, sheet_name='Dataset1')
# organisation_ids = df['Organisation ID'].tolist()
view_id = '23837774' #23837774 #131790917234
dimension_name = 'landingPagePath'
metric_name = 'sessions'

# CHANGE THE PATH AND DATE
# excel_path = '/content/21-22.xlsx'
start_date = '2021-07-01'
end_date = '2021-07-01'

organisation_ids = [236701]
# DO NOT MODIFY
for org_id in organisation_ids:
    print('Org id ', org_id)
    try:
      org_id = str(int(org_id))
    except:
      break

    response = get_google_analytics_data(view_id, start_date, end_date, dimension_name, metric_name, org_id)
    print('response ', response)

    if response:
        for row in response:
            data.append({'Organisation ID': org_id, 'Landing Page Path': row['dimensions'][0], 'Sessions': row['metrics'][0]['values'][0]})
    else:
        data.append({'Organisation ID': org_id, 'Landing Page Path': 'None', 'Sessions': 'None'})

print('data ', data)
# data_df = pd.DataFrame(data)
# data_df.head()
# output_excel_path = '/content/analytics_data.xlsx'
# data_df.to_excel(output_excel_path, index=False)

# print("Data exported to:", output_excel_path)